In [2]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)

def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 

class Node:
    question_number = -1
    follow_up_number = 0
    raw_question = ""
    shortened_question = ""
    references = []
    instructor_answer = ""
    student_answer = ""
    question_folders = []
    
    
    def __init__(self, question_number, follow_up_number, raw_question, shortened_question, instructor_answer, student_answer, question_folders):
        self.question_number = question_number
        self.raw_question = raw_question
        self.follow_up_number = follow_up_number
        self.raw_question = raw_question
        self.instructor_answer = instructor_answer
        self.student_answer = student_answer
        self.shortened_question = shortened_question
        self.question_folders = question_folders
        
    def __str__(self):
        return self.question_folder + " " + self.shortened_question
    
    def __repr__(self):
        return self.question_folder + " " + self.shortened_question
    
    def add_references(self, all_refs):
        self.references = all_refs
        
    def add_shortened_questions(self, shortened_question):
        self.shortened_question = shortened_question

In [3]:
import nltk
from nltk.tokenize import word_tokenize
import string 

def shortenQuestions(stop_words, question):

    shortened_questions = []

    tokens = word_tokenize(question)
    tokens = [w.lower() for w in tokens]
    words = [w for w in tokens if (not w in stop_words and w != "" and w != " ")]

    # Get rid of punctuation
    table = str.maketrans('', '', string.punctuation)
    words = [w.translate(table) for w in words]

    final_question = " ".join(words)
    return final_question

def createStopWords(questions):
        
    stop_words = ["ourselves", "hers", "between", "yourself", "but", "again", 
                      "there", "about", "once", "during", "out", "very", "having", 
                      "with", "they", "own", "an", "be", "some", "for", "do", "its", 
                      "yours", "such", "into", "of", "most", "itself", "other", "off",
                      "is", "s", "am", "or", "who", "as", "from", "him", "each", "the",
                      "themselves", "until", "below", "are", "we", "these", "your", "his", 
                      "through", "don", "nor", "me", "were", "her", "more", "himself", "this",
                      "down", "should", "our", "their", "while", "above", "both", "up", "to",
                      "ours", "had", "she", "all", "no", "when", "at", "any", "before", "them",
                      "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", 
                      "then", "that", "because", "what", "over", "why", "so", "can", "did", 
                      "not", "now", "under", "he", "you", "herself", "has", "just",
                      "where", "too", "only", "myself", "which", "those", "i", "after",
                      "few", "whom", "t", "being", "if", "theirs", "my", "against", "a",
                      "by", "doing", "it", "how", "further", "was", "here", "than", "ok",
                      "thank", "ah"]
        
    entireTextTokens = nltk.tokenize.word_tokenize(questions)
    
    # Get rid of punctuation
    table = str.maketrans('', '', string.punctuation)
    entireTextTokens = [w.translate(table) for w in entireTextTokens]

    entireTextDist = nltk.FreqDist(w.lower() for w in entireTextTokens if w and w not in stop_words)

    mostCommon = entireTextDist.most_common(75)

    for i in mostCommon:
        stop_words.append(i[0])
        print(i)
        
    return stop_words

In [4]:
import csv
import datetime

def textFromCSV(filename):

    # reading csv file 
    with open(filename, 'r', encoding="utf8") as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 

        # extracting field names through first row 
        fields = next(csvreader) 

        rows = []

        # extracting each data row one by one 
        for row in csvreader: 
            rows.append(row) 
        
        return rows, fields

In [11]:
def parseCSV(filename):
    
    rows, fields = textFromCSV(filename)
    entireText = ""

    curr_text = ""
    curr_post_number = "";
    curr_folder = ""
    curr_timestamp = ""
    final_column_question = ""
    final_column_instructor_answer = ""
    final_column_student_answer = ""
    all_contributions = {}
    raw_references = {}
    all_folders = []
    folders = []
    raw_timestamps_to_hw = {}

    for row in rows[:]: 
        # print(row)
        # parsing each column of a row 
        for col,field in zip(row, fields): 
            if(field == "Post Number"):
                if not curr_post_number == "" :
                    entireText = entireText + " " + final_column_question
                    node = Node(curr_post_number, 0, final_column_question, "", final_column_instructor_answer, final_column_student_answer, folders)
                    all_contributions[(str(curr_post_number), 0)] = node
                    
                    if curr_timestamp[-3:] == 'UTC':
                        curr_timestamp = curr_timestamp[:-4]
                    timestamp = datetime.datetime.strptime(curr_timestamp, '%Y-%m-%d %H:%M:%S')
                    
                    for folder in folders:
                        hw_timestamps = []
                        if folder in raw_timestamps_to_hw:
                            hw_timestamps = raw_timestamps_to_hw[folder]

                        hw_timestamps.append(timestamp)
                        raw_timestamps_to_hw.update({folder: hw_timestamps})
                
                    for loc in find_all(final_column_instructor_answer, "@"):
                        next_space = loc + 1
                        while(next_space <= len(final_column_instructor_answer)-1 and final_column_instructor_answer[next_space].isdigit()):
                            next_space = next_space + 1

                        question = -1
                        try:
                            question = int(final_column_instructor_answer[(loc + 1):next_space])
                        except ValueError:
                            question = -1
                        if curr_post_number in raw_references:
                            raw_references_q = raw_references[curr_post_number]
                            raw_references_q.append(question)
                            raw_references.update({curr_post_number : raw_references_q})
                        else:
                            raw_references_q = []
                            raw_references_q.append(question)
                            raw_references.update({curr_post_number : raw_references_q})

                        if question in raw_references:
                            raw_references_q = raw_references[question]
                            raw_references_q.append(curr_post_number)
                            raw_references.update({str(question) : raw_references_q})
                        else:
                            raw_references_q = []
                            raw_references_q.append(curr_post_number)
                            raw_references.update({str(question) : raw_references_q})
                curr_post_number = col

            elif(field == "Submission HTML Removed"): #the question could also be in subject
                curr_text = col

            elif(field == "Folders"):
                curr_folder = col

                folders = []
                curr_folders = curr_folder.split()
                for folder in curr_folders:
                    new_folder = folder
                    if folder[-1] == ';':
                        new_folder = folder[:-1]
                    folders.append(new_folder)

                    if new_folder not in all_folders:
                        all_folders.append(new_folder) 

            elif(field == "Created At"):
                curr_timestamp = col

            elif(field == "Part of Post"):
                if(col == "started_off_question"):
                    final_column_question = curr_text

                elif(col == "updated_question"):
                    final_column_question = curr_text

                elif(col == "updated_i_answer"):
                    final_column_instructor_answer = curr_text

                elif(col == "started_off_i_answer"):
                    final_column_instructor_answer = curr_text

                elif(col == "updated_s_answer"):
                    final_column_student_answer = curr_text

                elif(col == "started_off_s_answer"):
                    final_column_student_answer = curr_text

                elif(col == "followup"):
                    continue

                elif(col == "reply_to_followup"):
                    continue

    # raw_references_sorted = sorted(raw_references.items(), key=lambda v: len(v[1]))

    stop_words = createStopWords(entireText)
    for key in all_contributions:
        shortened_question = shortenQuestions(stop_words, all_contributions[key].raw_question)
        all_contributions[key].add_shortened_questions(shortened_question)
    
    for key, value in raw_references.items():
        without_duplicates = Remove(value)
        raw_references.update({key: without_duplicates})

    for key in raw_references:
        if (str(key),0) in raw_references:
            all_contributions[(str(key),0)].add_references(raw_references[key])

    for key in all_contributions:
        print(key)
        contribution = all_contributions[key]
        print("Question Number: %s \n Question: %s \n Shortened Question: %s \n Answer: %s \n %s \n" % \
              (contribution.question_number, contribution.raw_question, contribution.shortened_question, contribution.instructor_answer, contribution.references))
        
    return all_contributions, entireText, raw_timestamps_to_hw, all_folders

In [12]:
parseCSV('contributions.csv')

('i', 16238)
('gt', 3853)
('nt', 3609)
('lt', 3397)
('n', 3300)
('1', 2775)
('would', 2443)
('m', 2304)
('thanks', 2128)
('class', 2115)
('is', 2023)
('time', 1939)
('2', 1907)
('node', 1825)
('use', 1798)
('error', 1777)
('method', 1709)
('algorithm', 1659)
('one', 1611)
('o', 1611)
('if', 1547)
('get', 1518)
('test', 1492)
('graph', 1450)
('return', 1394)
('like', 1350)
('make', 1324)
('know', 1319)
('for', 1307)
('the', 1263)
('can', 1234)
('null', 1214)
('could', 1190)
('need', 1179)
('list', 1177)
('case', 1169)
('page', 1164)
('sure', 1149)
('since', 1133)
('b', 1116)
('string', 1107)
('set', 1091)
('what', 1066)
('0', 1063)
('question', 1063)
('3', 1059)
('two', 1057)
('code', 1046)
('also', 1016)
('value', 1011)
('tree', 1006)
('example', 999)
('new', 980)
('first', 977)
('in', 975)
('file', 970)
('array', 958)
('find', 957)
('say', 955)
('runtime', 950)
('hi', 930)
('v', 922)
('way', 904)
('confused', 899)
('mean', 878)
('when', 855)
('element', 844)
('run', 842)
('size', 835)

Question Number: 1062 
 Question: The solution says the TAs are the men and the pierogi stations are the women. Does it matter for this algorithm whether TAs propose to the stations or the stations propose to the TAs? If it does matter, how does this affect the algorithm?  
 Shortened Question: solution says tas men pierogi stations women  matter whether tas propose stations stations propose tas  matter  affect  
 Answer: We accepted answers for both, as long as they had the correct preference lists. Our only goal is to produce a stable matching, so it's not important which represents the men/women. 
 [] 

('1063', 0)
Question Number: 1063 
 Question: We offered in lecture facts similar to the formula below. Can we use this formula without justification on the exam?

$$\sum_{k = 1}^n \Theta(f(k)) = \Theta(\sum_{k = 1}^n f(k))$$ 
 Shortened Question: offered lecture facts similar formula  formula without justification exam    sum  k   n theta  f  k    theta  sum  k   n f  k     
 Answer

('1764', 0)
Question Number: 1764 
 Question: Hello, 

I saw that the autograder results for MS1 were released. I ended up failing two of the tests, but I don't understand what the failures mean. This is what they each said:

WikiXmlDumpParserImpl: parse two pages (0.0/2.0)
Test Failed!Not true that in neighbors of &lt;edu.upenn.cis121.project.impl.Page@46ae223&gt; (&lt;[]&gt;) contains exactly &lt;[DummyId{id='April'}]&gt;. It is missing &lt;[DummyId{id='April'}]&gt;    at edu.upenn.cis121.project.Milestone1CompilationTest.parseTwoWikiPage:119 (Milestone1CompilationTest.java)
WikiXmlDumpParser: forward links to unseen pages (0.0/4.0)
Test Failed!Not true that in neighbors of &lt;edu.upenn.cis121.project.impl.Page@51&gt; (&lt;[]&gt;) contains exactly &lt;[edu.upenn.cis121.project.impl.Page@1f7af1]&gt;. It is missing &lt;[edu.upenn.cis121.project.impl.Page@1f7af1]&gt;    at edu.upenn.cis121.project.Milestone1CompilationTest.testForwardReferences:142 (Milestone1CompilationTest.java)

As 


('2758', 0)
Question Number: 2758 
 Question: How do we extract the type of edgeweight. is there an e.getType() method that returns an enumerated type?  
 Shortened Question: extract type edgeweight  egettype   returns enumerated type  
 Answer: Please email your recitation TA 
 [] 

('2759', 0)
Question Number: 2759 
 Question: The write-up says the following:

You will need to split text into constituent words. For our tests, you need only to split the text on whitespace, like by str.split("\\s+"). For simplicity, you can lowercase all characters and throw out all characters that are not in the 26 letters of the English alphabet. So some text like, “Isn’t 17 an awesome-great number?” will produce the words “isnt”, “an”, “awesomegreat”, “number”, but with no mapping for “17” because “17” does not have any lowercase English letters. Note that you might want to improve upon this na¨ıve word tokenization for extra credit.

So do we treat the input text to search in the same way? Like fo

AttributeError: 'Node' object has no attribute 'question_folder'